In [1]:
import pandas as pd

In [2]:
car = pd.read_csv('data/quikr_car.csv')

In [3]:
car.head()

name   company  year          Price  \
0    Hyundai Santro Xing XO eRLX Euro III   Hyundai  2007         80,000   
1                 Mahindra Jeep CL550 MDI  Mahindra  2006       4,25,000   
2              Maruti Suzuki Alto 800 Vxi    Maruti  2018  Ask For Price   
3  Hyundai Grand i10 Magna 1.2 Kappa VTVT   Hyundai  2014       3,25,000   
4        Ford EcoSport Titanium 1.5L TDCi      Ford  2014       5,75,000   

   kms_driven fuel_type  
0  45,000 kms    Petrol  
1      40 kms    Diesel  
2  22,000 kms    Petrol  
3  28,000 kms    Petrol  
4  36,000 kms    Diesel

In [4]:
car.shape

(892, 6)

In [5]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


### Quality
- year has many non-year values
- year is object not integer
- price has "Ask for price"
- price is object not integer
- kms_driven has kms with integer
- kms_driven is object not integer
- kms_driven has NaN values
- fuel_type has NaN values
- keep first 3 words of name

### Cleaning

In [6]:
backup=car.copy()

In [7]:
car=car[car['year'].str.isnumeric()]

In [8]:
car['year']=car['year'].astype(int)

In [9]:
car=car[car['Price'] != 'Ask For Price']

In [10]:
car['Price'].head(5)

0      80,000
1    4,25,000
3    3,25,000
4    5,75,000
6    1,75,000
Name: Price, dtype: object

In [11]:
car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 819 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        819 non-null    object
 1   company     819 non-null    object
 2   year        819 non-null    int64 
 3   Price       819 non-null    object
 4   kms_driven  819 non-null    object
 5   fuel_type   816 non-null    object
dtypes: int64(1), object(5)
memory usage: 44.8+ KB


In [12]:
car['Price']=car['Price'].str.replace(',','').astype(int)
car['Price'].head()

0     80000
1    425000
3    325000
4    575000
6    175000
Name: Price, dtype: int64

In [13]:
car['kms_driven']=car['kms_driven'].str.split(" ").str.get(0).str.replace(",","")

In [14]:
car=car[car['kms_driven'].str.isnumeric()]

In [15]:
car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 817 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        817 non-null    object
 1   company     817 non-null    object
 2   year        817 non-null    int64 
 3   Price       817 non-null    int64 
 4   kms_driven  817 non-null    object
 5   fuel_type   816 non-null    object
dtypes: int64(2), object(4)
memory usage: 44.7+ KB


In [16]:
car['kms_driven'] = car['kms_driven'].astype(int)

In [17]:
car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 817 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        817 non-null    object
 1   company     817 non-null    object
 2   year        817 non-null    int64 
 3   Price       817 non-null    int64 
 4   kms_driven  817 non-null    int64 
 5   fuel_type   816 non-null    object
dtypes: int64(3), object(3)
memory usage: 44.7+ KB


In [18]:
car=car[~car['fuel_type'].isna()]

In [19]:
car['name']

0        Hyundai Santro Xing XO eRLX Euro III
1                     Mahindra Jeep CL550 MDI
3      Hyundai Grand i10 Magna 1.2 Kappa VTVT
4            Ford EcoSport Titanium 1.5L TDCi
6                                   Ford Figo
                        ...                  
883                Maruti Suzuki Ritz VXI ABS
885                 Tata Indica V2 DLE BS III
886                      Toyota Corolla Altis
888                       Tata Zest XM Diesel
889                        Mahindra Quanto C8
Name: name, Length: 816, dtype: object

In [20]:
car['name']=car['name'].str.split(" ").str.slice(0,3).str.join(" ")

In [21]:
car=car.reset_index(drop=True)

In [22]:
car.describe()

year         Price     kms_driven
count   816.000000  8.160000e+02     816.000000
mean   2012.444853  4.117176e+05   46275.531863
std       4.002992  4.751844e+05   34297.428044
min    1995.000000  3.000000e+04       0.000000
25%    2010.000000  1.750000e+05   27000.000000
50%    2013.000000  2.999990e+05   41000.000000
75%    2015.000000  4.912500e+05   56818.500000
max    2019.000000  8.500003e+06  400000.000000

In [23]:
car=car[car['Price'] < 6e6].reset_index(drop=True)

In [24]:
car.to_csv('data/cleaned_quikr_car.csv', index=False)

### Model

In [25]:
X= car.drop(columns=['Price'])
y = car['Price']

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

In [28]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [29]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [30]:
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'), ['name','company','fuel_type']),
                                       remainder='passthrough')

In [31]:
lr = LinearRegression()

In [32]:
pipe=make_pipeline(column_trans, lr)

In [33]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [34]:
y_pred=pipe.predict(X_test)


In [35]:
r2_score(y_test,y_pred)

0.5213705523729396

In [36]:
import numpy as np
scores = []
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=i)
    lr = LinearRegression()
    pipe=make_pipeline(column_trans, lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))




In [37]:
np.argmax(scores)

np.int64(433)

In [38]:
scores[np.argmax(scores)]

0.8457059008496111

In [39]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))
lr = LinearRegression()
pipe=make_pipeline(column_trans, lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8457059008496111

In [40]:
import pickle
pickle.dump(pipe, open('LinearRegressionModel.pkl','wb'))

In [41]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti', 2019,100,'Petrol']],columns=['name','company','year','kms_driven','fuel_type']))

array([458894.11060714])